<a href="https://colab.research.google.com/github/nitishymtpl/innovation-agri-hackathon/blob/MTPL-Challenge5/Load_and_route_optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In load and route optimisation problems, many different transportation tools such as trucks, airplanes and ships are available. Different choices of routes and transporation tools will lead to different costs. To minimize cost, we should consider goods consolidation (Occassions when different goods share a journey together.), different transportation costs and delivery time constraints etc. This project uses mathematical programming to model such situation and solves for overall cost minimization solution. The model construction offers options of mathematical programming fremwork using CVXPY. CVXPY is an opensource programming framework originally developed by a Stanford research team.

**Problem Statement:** In our simulated case, there are 8 goods, 4 arbitary/dummy cities/countries (Gujarat, Mumbai, Singapore, Malaysia), 16 ports and 4 transportation tools. The 8 goods originate from different cities and have different destinations. Each city/country has 4 ports, the airport, railway station, seaport and warehouse. There are in total 50 direct routes connecting different ports. **Please check model.xlsx<data file>.** Each route has a specific transportation tool, transportation cost, transit time and weekly schedule. Warehouse in each city allows goods to be deposited for a period of time so as to fit certain transportation schedules or wait for other goods to be transported together. All goods might have different order dates and different delivery deadlines. With all these criteria, how can we find out solution routes for all goods that minimize the overall operational cost with optimal distribution of load and route?

In [ ]:
!pip install cplex
!pip install docplex
!pip install cylp
!pip install osmnx
!pip install networkx
from itertools import product
import numpy as np
import cvxpy as cp
import pandas as pd
import json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43.3 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 610 kB 9.0 MB/s 
  Created wheel for docplex: filename=docplex-2.23.222-py3-none-any.whl size=662847 sha256=302f434cc6c38e94763a40e5664782bab2dbb5fbbe79ee7e9ca43d8cad0cdc24
  Stored in directory: /root/.cache/pip/wheels/a7/c9/fb/cee5a89f304e77a39c466e625ac2830434b76eb8384999d116
Successfully built docplex
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.2 MB 6.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 95 kB 3.0 MB/s 
     |████████████████████████████████| 6.3 MB 16.2 MB/s 
     |██████████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Assumptions**:

Before model building, some assumptions should be made to simplify the case because real-world delivery problems consist of too many unmeasurable factors that can affect the delivery process and final outcomes. Here are the main assumptions:
1. The delivery process is deterministic, no random effect will appear on delivery time and cost etc.
2. Goods can be transported in normal container, no special containers (refrigerated, thermostatic etc.) will be needed.
3. Container only constraints on the good's volume, and all goods are divisible in terms of volume. (No bin packing problem needed to be considered.)
4. The model only evaluates the major carriage routes. The first and last mile between end user and origin/destination shipping point are not considered. (From warehouse to warehouse.)
5. There is only one transportation tool available between each two ports. For instance, we can only directly go from one airport to the other airport in different cities by flight, while direct journey by ship or railway or truck is infeasible.
6. Overall cost is restricted to the most important 3 parts, transportation cost, warehouse cost and goods tariff.
7. The minimum unit for time is day in the model, and there is at most one transit in a route in one day.

**Dimension \& Matrixing**:

In order to make the criteria logic clearer and the calculation more efficient, we use the concept of matrixing to build the necessary components in the model. In our case, there are totally 4 dimensions:
1. Start Port: $i$
Indicating the start port of a direct transport route. The dimension length equals the total number of ports in the data.
2. End Port: $j$
Indicating the end port of a direct transport route. The dimension length equals the total number of ports in the data.
3. Time: $\boldsymbol{t}$
Indicating the departure time of a direct transport. The dimension length equals the total number of days between the earliest order date and the latest delivery deadline date of all goods in the data.
4. Goods: $k$
Indicating the goods to be transported. The dimension length equals the total number of goods in the data.
All the variable or parameter matrices to be introduced in the later parts will have one or more of these 4 dimensions.

**Decision Variables**:

As mentioned above, we will use the concept of variable matrix, a list of variables deployed in the form of a matrix or multi-dimensional array. In our model, 3 variable matrices will be introduced:
1. Decision Variable Matrix: $X$

The most important variable matrix in the model. It's a 4 dimensional matrix, each dimension representing start port, end port, time and goods respectively. Each element in the matrix is a binary variable, representing whether a route is taken by a specific goods. For example, element $X_{i, j, t, k}$ represents whether goods $k$ travels from port i to port $j$ at time $t$.
2. Container Number Matrix: $Y$

A variable matrix used to support the decision variable matrix. It's a 3 dimensional matrix, with each dimension representing start port, end port and time respectively. Each element in the matrix is an integer variable, representing the number of containers needed in a specific route. For example, $Y_{i, j, t}$ represents the number of containers needed to load all the goods travelling simultaneously from port i to port $j$ at time $t$. Such matrix is introduced to make up for the limitation of "linear operator only" in mathematical programming, when we need a roundup() method in direct calculation of the container number.
3. Route Usage Matrix: $Z$

A variable matrix used to support the decision variable matrix. It's a 3 dimensional matrix, with each dimension representing start port, end port and time respectively. Each element in the matrix is a binary variable, representing whether a route is used or not. For instance, $Z_{i, j, t}$ represents whether the route from port $i$ to port $j$ at time $t$ is used or not (no matter which goods). It's introduced with similar purpose to $Y_{i, j, t}$.


**Parameters**:

Similar to the decision variables, the following parameter arrays or matrices are introduced for the sake of later model building:
1. Per Container Cost: $\quad$ C

A 3 dimensional parameter matrix, each dimension representing start port, end port and time. $C_{i, j, t}$ in the matrix represents the overall transportation cost per container from port $i$ to port $j$ at time $t$. This overall cost includes handling cost, bunker/fuel cost, documentation cost, equipment cost and extra cost from model data.xlsx. For infeasible route, the cost element will be set to be big $\mathrm{M}$ (an extremely large number), making the choice infeasible.
2. Route Fixed Cost: $\quad$ FC

A 3 dimensional parameter matrix, each dimension representing start port, end port and time. $F C_{i, j, t}$ in the matrix represents the fixed transportation cost to travel from port $i$ to port $j$ at time $t$, regardless of goods number or volume. For infeasible route, the cost element will be set to be big $M$ as well.
3. Warehouse Cost: $\quad$ wh

A one dimension array with dimension start port. $w h_{i}$ represents the warehouse cost per cubic meter per day at port i. Warehouse cost for ports with no warehouse function (like airport, railway station etc.) is set to be big M.
4. Transportation Time: $T$

A 3 dimensional parameter matrix, each dimension representing start port, end port and time. $T_{i j, t}$ in the matrix represents the overall transportation time from port $i$ to port $j$ at time $t$. This overall time includes custom clearance time, handling time, transit time and extra time from model data.xlsx. For infeasible route, the time element will be set to be big $M$.
5. Tax Percentage: tax

A one dimension array with dimension goods. tax ${ }_{k}$ represents the tax percentage for goods $k$ imposed by its destination country. If the goods only goes through a domestic transit, the tax percentage for such goods will be set as $0 .$
6. Transit Duty: $td$ 

A two dimensional matrix, each dimension representing start port and end port. $t d_{i, j}$ represents the transit duty (tax imposed on goods passing through a country) percentage for goods to go from port $i$ to port $j$. If port i and port $j$ belong to the same country, transit duty percentage is set to be 0 . For simplicity purpose, transit duty is set to be equal among all goods. (can be extended easily)
7. Container Volume: $ctnV$

A two dimensional matrix, each dimension representing start port and end port. $c t n V_{i, j}$ represents the volume of container in the route from port $i$ to port $j$.
8. Goods Volume: $V$

A one dimension array with dimension goods. $V_{k}$ represents the volume of goods $k$.
9. Goods Value: val

10. Order Date: ord

A one dimension array with dimension goods. ord $k$ represents the order date of goods $k$.
11. Deadline Date: $d d l$

A one dimension array with dimension goods. $d d l_{k}$ represents the deadline delivery date of goods $k$.
12. Origin Port: $O P$

A one dimension array with dimension goods. $O P_{k}$ represents the port where goods $\mathrm{k}$ starts from.
13. Destination Port: $D P$

A one dimension array with dimension goods. $D P_{k}$ represents the port where goods $k$ ends up to be in.
The data of all the above parameter matrices will be imported from model data.xIsx with function transform() and set_param().


**Mathematical Modelling**:

With all the variables and parameters defined above, we can build up the objectives and constraints to form an integer programming model.
Objective
The objective of the model is to minimize the overall cost, which includes 3 parts, transportation cost, warehouse cost and tax cost. Firstly, the transportation cost includes container cost and route fixed cost. Container cost equals the number of containers used in each route times per container cost while route fixed cost equals the sum of fixed cost of all used routes. Secondly, the warehouse cost equals all goods' sum of volume times days of storage times warehouse fee per cubic meter per day in each warehouse. Finally, the tax cost equals the sum of import tariff and transit duty of all goods. 

Mathematic formulation are attached below.

$\min _{x, y, z}$ Transportation Cost $+$ Warehouse Cost $+$ Tax Cost

Transportation Cost $=\sum_{t} \sum_{j} \sum_{i} Y_{i, j, t} \times C_{i, j, t}+Z_{i, j, t} \times F C_{i, j, t}$

Warehouse Cost $=\sum_{i}\left[\sum_{k}\left(\sum_{j} \sum_{t} t \times x_{i, j, t, k}\right) \times V_{k}\right] \times w h_{i}-\sum_{j}\left\{\sum_{k}\left[\sum_{i} \sum_{t}\left(t+T_{i, j, t}\right) \times x_{i, j, t, k}\right] \times V_{k}\right\} \times w h_{j}$
$$
\operatorname{Tax} \text { Cost }=\sum_{k} \operatorname{tax}_{k} * \operatorname{val}_{k}+\sum_{i} \sum_{j}\left(\sum_{t} \sum_{k} x_{i, j, t, k} \times v a l_{k}\right) \times t d_{i, j}
$$


**Constraints**:

1. For each goods $k$, it must be shipped out from its origin to another node and shipped to its destination.
$$
\begin{aligned}
&\sum_{t} \sum_{j} x_{O P_{k}, j, t, k}=1, \forall k \\
&\sum_{t} \sum_{i} x_{i, D P_{k}, t, k}=1, \forall k
\end{aligned}
$$

2. For each goods $k$, it couldn't be shipped out from its destination or shipped to its origin.
$$
\begin{aligned}
&\sum_{t} \sum_{i} x_{i, 0 P_{k}, t, k}=0, \forall k \\
&\sum_{t} \sum_{j} x_{D P_{k}, j, t, k}=0, \forall k
\end{aligned}
$$

3. For each goods $k$ at transition point $j$ (neither origin nor destination), ship-in times must equal ship-out times.
$$
\sum_{t} \sum_{i} x_{i, j, t, k}=\sum_{t} \sum_{i} x_{j, i, t, k}, \forall k, j \text { s.t. } j \neq O P_{k}, j \neq D P_{k}
$$

4. Each goods k can only be transitioned in or out of a port for at most once.
$$
\begin{aligned}
&\sum_{t} \sum_{j} x_{i, j, t, k} \leq 1, \forall k, i \\
&\sum_{t} \sum_{i} x_{i, j, t, k} \leq 1, \forall k, j
\end{aligned}
$$

5. For each goods $k$ at transition port $j$, ship-out time should be after ship-in time. For goods $k$ at its origin port, ship-out time should be after order date. (Or stay time greater than order date, because ship-in time is none)
$$
\begin{aligned}
&\sum_{j} \sum_{t} t \times x_{i, j, t, k}-\sum_{j} \sum_{t}\left(t \times x_{j, i, t, k}+T_{j, i, t} \times x_{j, i, t, k}\right) \geq 0, \forall k, i \text { s.t. } i \neq O P_{k}, i \neq D P_{k} \\
&\sum_{j} \sum_{t} t \times x_{O P_{k}, j, t, k}-\sum_{j} \sum_{t}\left(t \times x_{j, O P_{k}, t, k}+T_{j, O P_{k, t}} \times x_{j, O P_{k}, t, k}\right) \geq \operatorname{ord}_{k}, \forall k
\end{aligned}
$$

6. At each route at time $t$, the total volume of containers should be larger than the total volume of goods.
$$
y_{i, j, t} \geq \frac{\sum_{k} x_{i, j, t, k} \times V_{k}}{\operatorname{ctn} V_{i, j}}, \forall i, j, t
$$

7. Check whether a route is used at time t. Because $Z_{i, j, t}$ is binary variable, if a route is used, sum of $X_{i, j, t, k}$ for all goods $k$ at $i, j, t$ must be greater than 0 . We can scale it back to $[0,1]$ by multiplying a small number.
$$
z_{i, j, t} \geq \sum_{k} x_{i, j, t, k} \times 10^{-5}, \forall i, j, t
$$

8. For each goods $k$, it should be shipped to its destination port before the deadline delivery date.
$$
\sum_{i} \sum_{t}\left(t \times x_{i, D P_{k}, t, k}+T_{i, D P_{k}, t} \times x_{i, D P_{k}, t, k}\right) \leq d d l_{k}, \forall k
$$





In [ ]:
class SCO:
    '''a Model class that solves the supply chain transportation optimization problem.'''

    def __init__(self, framework='CVXPY'):
        # parameters
        self.port_Space = None
        self.date_Space = None
        self.Goods = None
        self.index_Port = None
        self.port_Index = None
        self.max_Date = None
        self.min_Date = None
        self.tran_Cost = None
        self.tran_Fixed_Cost = None
        self.tran_Time = None
        self.container_Vol = None
        self.wh_Cost = None
        self.k_Vol = None
        self.k_Value = None
        self.k_DDL = None
        self.k_Start_Port = None
        self.k_End_Port = None
        self.k_Start_Time = None
        self.tax_Pct = None
        self.transit_Duty = None
        self.route_num = None
        self.available_routes = None
        # decision variables
        self.var = None
        self.x = None
        self.var_2 = None
        self.y = None
        self.var_3 = None
        self.z = None
        # result & solution
        self.xs = None
        self.ys = None
        self.zs = None
        self.wh_Cost_Final = None
        self.transport_Cost = None
        self.tax_Cost = None
        self.solution_ = None
        self.arrTime_ = None
        self.objective_value = None
        # helping variables
        self.var_location = None
        self.var_2_location = None
        self.var_3_location = None



    def set_param(self, route, order):
        '''set model parameters based on the read-in route and order information.'''

        bigM = 100000
        route = route[route['Feasibility'] == 1]
        route['Warehouse Cost'][route['Warehouse Cost'].isnull()] = bigM
        route = route.reset_index()

        portSet = set(route['Source']) | set(route['Destination'])

        self.port_Space = len(portSet)
        self.port_Index = dict(zip(range(len(portSet)), portSet))
        self.index_Port = dict(zip(self.port_Index.values(), self.port_Index.keys()))

        self.max_Date = np.max(order['Required Delivery Date'])
        self.min_Date = np.min(order['Order Date'])
        self.date_Space = (self.max_Date - self.min_Date).days
        startWeekday = self.min_Date.weekday() + 1
        weekday = np.mod((np.arange(self.date_Space) + startWeekday), 7)
        weekday[weekday == 0] = 7
        weekdayDateList = {i: [] for i in range(1, 8)}
        for i in range(len(weekday)):
            weekdayDateList[weekday[i]].append(i)
        for i in weekdayDateList:
            weekdayDateList[i] = json.dumps(weekdayDateList[i])

        source = list(route['Source'].replace(self.index_Port))
        destination = list(route['Destination'].replace(self.index_Port))
        DateList = list(route['Weekday'].replace(weekdayDateList).apply(json.loads))

        self.Goods = order.shape[0]
        self.tran_Cost = np.ones([self.port_Space, self.port_Space, self.date_Space]) * bigM
        self.tran_Fixed_Cost = np.ones([self.port_Space, self.port_Space, self.date_Space]) * bigM
        self.tran_Time = np.ones([self.port_Space, self.port_Space, self.date_Space]) * bigM

        for i in range(route.shape[0]):
            self.tran_Cost[source[i], destination[i], DateList[i]] = route['Cost'][i]
            self.tran_Fixed_Cost[source[i], destination[i], DateList[i]] = route['Fixed Freight Cost'][i]
            self.tran_Time[source[i], destination[i], DateList[i]] = route['Time'][i]

        self.transit_Duty = np.ones([self.port_Space, self.port_Space]) * bigM
        self.transit_Duty[source, destination] = route['Transit Duty']

        # make the container size of infeasible routes to be small enough, similar to bigM
        self.container_Vol = np.ones([self.port_Space, self.port_Space]) * 0.1
        self.container_Vol[source, destination] = route['Container Size']
        self.container_Vol = self.container_Vol.reshape(self.port_Space, self.port_Space, 1)
        self.wh_Cost = route[['Source', 'Warehouse Cost']].drop_duplicates()
        self.wh_Cost['index'] = self.wh_Cost['Source'].replace(self.index_Port)
        self.wh_Cost = np.array(self.wh_Cost.sort_values(by='index')['Warehouse Cost'])
        self.k_Vol = np.array(order['Volume'])
        self.k_Value = np.array(order['Order Value'])
        self.k_DDL = np.array((order['Required Delivery Date'] - self.min_Date).dt.days)
        self.k_Start_Port = np.array(order['Ship From'].replace(self.index_Port))
        self.k_End_Port = np.array(order['Ship To'].replace(self.index_Port))
        self.k_Start_Time = np.array((order['Order Date'] - self.min_Date).dt.days)
        self.tax_Pct = np.array(order['Tax Percentage'])

        # add available route indexes
        self.route_num = route[['Source', 'Destination']].drop_duplicates().shape[0]
        routes = route[['Source', 'Destination']].drop_duplicates().replace(self.index_Port)
        self.available_routes = list(zip(routes['Source'], routes['Destination']))
        # localization variables of decision variables in the matrix
        var_location = product(self.available_routes, range(self.date_Space), range(self.Goods))
        var_location = [(i[0][0], i[0][1], i[1], i[2]) for i in var_location]
        self.var_location = tuple(zip(*var_location))

        var_2_location = product(self.available_routes, range(self.date_Space))
        var_2_location = [(i[0][0], i[0][1], i[1]) for i in var_2_location]
        self.var_2_location = tuple(zip(*var_2_location))

        self.var_3_location = self.var_2_location

    def build_model(self):
        '''overall function to build up model objective and constraints'''
        self.cvxpy_build_model()
        

    def cvxpy_build_model(self):
        '''build up the mathematical programming model's objective and constraints using CVXPY framework.'''

        # 4 dimensional binary decision variable matrix
        self.var = cp.Variable(self.route_num * self.date_Space * self.Goods, boolean=True, name='x')
        self.x = np.zeros((self.port_Space, self.port_Space, self.date_Space, self.Goods)).astype('object')
        self.x[self.var_location] = list(self.var)
        # 3 dimensional container number matrix
        self.var_2 = cp.Variable(self.route_num * self.date_Space, integer=True, name='y')
        self.y = np.zeros((self.port_Space, self.port_Space, self.date_Space)).astype('object')
        self.y[self.var_2_location] = list(self.var_2)
        # 3 dimensional route usage matrix
        self.var_3 = cp.Variable(self.route_num * self.date_Space, boolean=True, name='z')
        self.z = np.zeros((self.port_Space, self.port_Space, self.date_Space)).astype('object')
        self.z[self.var_3_location] = list(self.var_3)
        # warehouse related cost
        warehouseCost, arrTime, stayTime = self.warehouse_fee(self.x)
        ###objective###
        transport_Cost = np.sum(self.y * self.tran_Cost) + np.sum(self.z * self.tran_Fixed_Cost)
        transit_DutyCost = np.sum(np.sum(np.dot(self.x, self.k_Value), axis=2) * self.transit_Duty)
        tax_Cost = np.sum(self.tax_Pct * self.k_Value) + transit_DutyCost
        objective = cp.Minimize(transport_Cost + warehouseCost + tax_Cost)
        ###constraint###
        constraints = []
        # 1.Goods must be shipped out from its origin to another node and shipped to its destination.
        constraints += [np.sum(self.x[self.k_Start_Port[k], :, :, k]) == 1 for k in range(self.Goods)]
        constraints += [np.sum(self.x[:, self.k_End_Port[k], :, k]) == 1 for k in range(self.Goods)]
        # 2.For each Goods k, it couldn't be shipped out from its destination or shipped to its origin.
        constraints += [np.sum(self.x[:, self.k_Start_Port[k], :, k]) == 0 for k in range(self.Goods)]
        constraints += [np.sum(self.x[self.k_End_Port[k], :, :, k]) == 0 for k in range(self.Goods)]
        # 3.constraint for transition point
        for k in range(self.Goods):
            for j in range(self.port_Space):
                if (j != self.k_Start_Port[k]) & (j != self.k_End_Port[k]):
                    constraints.append(np.sum(self.x[:, j, :, k]) == np.sum(self.x[j, :, :, k]))
        # 4.each Goods can only be transitioned in or out of a port for at most once
        constraints += [np.sum(self.x[i, :, :, k]) <= 1 for k in range(self.Goods) for i in range(self.port_Space)]
        constraints += [np.sum(self.x[:, j, :, k]) <= 1 for k in range(self.Goods) for j in range(self.port_Space)]
        # 5.transition-out should be after transition-in
        constraints += [stayTime[j, k] >= 0 for j in range(self.port_Space) for k in range(self.Goods)]
        # 6.constraint for number of containers used
        numCtn = np.dot(self.x, self.k_Vol) / self.container_Vol
        constraints += [self.y[i, j, t] - numCtn[i, j, t] >= 0 \
                        for i in range(self.port_Space) for j in range(self.port_Space) for t in
                        range(self.date_Space) if not isinstance(self.y[i, j, t] - numCtn[i, j, t] >= 0, bool)]
        # 7. constraint to check whether a route is used
        constraints += [self.z[i, j, t] >= (np.sum(self.x[i, j, t, :]) * 10e-5) \
                        for i in range(self.port_Space) for j in range(self.port_Space) for t in
                        range(self.date_Space) if
                        not isinstance(self.z[i, j, t] >= (np.sum(self.x[i, j, t, :]) * 10e-5), bool)]
        # 8.time limitation constraint for each Goods
        constraints += [np.sum(arrTime[:, self.k_End_Port[k], :, k]) <= self.k_DDL[k] for k in range(self.Goods)
                        if not isinstance(np.sum(arrTime[:, self.k_End_Port[k], :, k]) <= self.k_DDL[k], bool)]
        model = cp.Problem(objective, constraints)

        self.objective = objective
        self.constraints = constraints
        self.model = model

    
    def solve_model(self, solver=cp.CBC):
        '''
        solve the optimization model & cache the optimized objective value, route and arrival time for each Goods.
        :param solver: the solver to use to solve the LP problem when framework is CVXPY, has no effect to the model
        Default solver is cvxpy.CBC, other open source solvers do not perform that well.
        :return: None
        '''
        try:
                self.objective_value = self.model.solve(solver)
                self.xs = np.zeros((self.port_Space, self.port_Space, self.date_Space, self.Goods))
                self.xs[self.var_location] = self.var.value
                self.ys = np.zeros((self.port_Space, self.port_Space, self.date_Space))
                self.ys[self.var_2_location] = self.var_2.value
                self.zs = np.zeros((self.port_Space, self.port_Space, self.date_Space))
                self.zs[self.var_3_location] = self.var_3.value

        except:
            raise Exception('Model is not solvable, no solution will be provided')

        nonzeroX = list(zip(*np.nonzero(self.xs)))
        nonzeroX = sorted(nonzeroX, key=lambda x: x[2])
        nonzeroX = sorted(nonzeroX, key=lambda x: x[3])
        nonzeroX = list(map(lambda x: (self.port_Index[x[0]], self.port_Index[x[1]], \
                                       (self.min_Date + pd.to_timedelta(x[2], unit='days')).date().isoformat(),
                                       x[3]), nonzeroX))

        self.wh_Cost_Final, arrTime, _ = self.warehouse_fee(self.xs)
        self.transport_Cost = np.sum(self.ys * self.tran_Cost) + np.sum(self.zs * self.tran_Fixed_Cost)
        self.tax_Cost = np.sum(self.tax_Pct * self.k_Value) + \
                       np.sum(np.sum(np.dot(self.xs, self.k_Value), axis=2) * self.transit_Duty)
        self.solution_ = {}
        self.arrTime_ = {}
        for i in range(self.Goods):
            self.solution_['Goods-' + str(i + 1)] = list(filter(lambda x: x[3] == i, nonzeroX))
            self.arrTime_['Goods-' + str(i + 1)] = (self.min_Date + pd.to_timedelta \
                (np.sum(arrTime[:, self.k_End_Port[i], :, i]), unit='days')).date().isoformat()

    def get_output_(self):
        '''After the model is solved, return total cost, final solution and arrival
        time for each of the Goods'''

        return self.objective_value, self.solution_, self.arrTime_

    def warehouse_fee(self, x):
        '''return warehouse fee, arrival time and stay time for each port.'''

        startTime = np.arange(self.date_Space).reshape(1, 1, self.date_Space, 1) * x
        arrTimeMtrx = startTime + self.tran_Time.reshape(self.port_Space, \
                                                        self.port_Space, self.date_Space, 1) * x
        arrTime = arrTimeMtrx.copy()
        arrTimeMtrx[:, self.k_End_Port.tolist(), :, range(self.Goods)] = 0
        stayTime = np.sum(startTime, axis=(1, 2)) - np.sum(arrTimeMtrx, axis=(0, 2))
        stayTime[self.k_Start_Port.tolist(), range(self.Goods)] -= self.k_Start_Time
        warehouseCost = np.sum(np.sum(stayTime * self.k_Vol, axis=1) * self.wh_Cost)

        return warehouseCost, arrTime, stayTime

    def txt_solution(self, route, order):
        '''transform the cached results to text.'''

        travelMode = dict(zip(zip(route['Source'], route['Destination']), route['Travel Mode']))
        txt = "Solution"
        txt += "\nNumber of Goods: " + str(order['Order Number'].count())
        txt += "\nTotal cost: " + str(self.transport_Cost + self.wh_Cost_Final + self.tax_Cost)
        txt += "\nTransportation cost: " + str(self.transport_Cost)
        txt += "\nWarehouse cost: " + str(self.wh_Cost_Final)
        txt += "\nTax cost: " + str(self.tax_Cost)

        for i in range(order.shape[0]):
            txt += "\n------------------------------------"
            txt += "\nGoods-" + str(i + 1) + "  Category: " + order['Commodity'][i]
            txt += "\nStart date: " + pd.to_datetime(order['Order Date']) \
                .iloc[i].date().isoformat()
            txt += "\nArrival date: " + str(self.arrTime_['Goods-' + str(i + 1)])
            txt += "\nRoute:"
            solution = self.solution_['Goods-' + str(i + 1)]
            route_txt = ''
            a = 1
            for j in solution:
                route_txt += "\n(" + str(a) + ")Date: " + j[2]
                route_txt += "  From: " + j[0]
                route_txt += "  To: " + j[1]
                route_txt += "  By: " + travelMode[(j[0], j[1])]
                a += 1
            txt += route_txt

        return txt


def transform(filePath):
    '''Read in order and route data, transform the data into a form that can
    be processed by the operation research model.'''
    order = pd.read_excel(filePath, sheet_name='Order Information')
    route = pd.read_excel(filePath, sheet_name='Route Information')
    order['Tax Percentage'][order['Journey Type'] == 'Domestic'] = 0
    route['Cost'] = route[route.columns[7:12]].sum(axis=1)
    route['Time'] = np.ceil(route[route.columns[14:18]].sum(axis=1) / 24)
    route = route[list(route.columns[0:4]) + ['Fixed Freight Cost', 'Time', \
                                              'Cost', 'Warehouse Cost', 'Travel Mode', 'Transit Duty'] + list(
        route.columns[-9:-2])]
    route = pd.melt(route, id_vars=route.columns[0:10], value_vars=route.columns[-7:] \
                    , var_name='Weekday', value_name='Feasibility')
    route['Weekday'] = route['Weekday'].replace({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, \
                                                 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7})

    return order, route

In [ ]:
from google.colab import drive

# Accessing My Google Drive, Please authorise it from gmail id. Just give the permission to access google drive file.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
if __name__ == '__main__':
    order, route = transform("/content/drive/My Drive/Colab Notebooks/data/model.xlsx") #Please upload the model.xlsx data file on your google drive location: My Drive/Colab Notebooks/data/model.xlsx
    m = SCO(framework='CVXPY') # for open source framework
    m.set_param(route, order)
    m.build_model()
    m.solve_model()
    txt = m.txt_solution(route, order)
    with open("/content/drive/My Drive/Colab Notebooks/data/Solution.txt", "w") as text_file:
        text_file.write(txt) #Please download the solution.txt file from your google drive file location: My Drive/Colab Notebooks/data/Solution.txt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:277: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


**Optimization Result \& Solution** :

With the objective \& constraints built, the model is now complete! To make users understand the result easier, we process it with the function txt_solution() and save it into a text file Solution.txt. The minimized cost value as well as optimal routes for all goods are presented in it.

route variable shows various available domestic and international route and cost associated with it. Please check model.xlsx file [Route information sheet].

In [ ]:
route

,Route Number,Source,Destination,Container Size,Fixed Freight Cost,Time,Cost,Warehouse Cost,Travel Mode,Transit Duty,Weekday,Feasibility
0,1,Singapore Port,Gujarat Port,67,300,8.0,900,NaN,Sea,0.002,1,1
1,2,Gujarat Port,Singapore Port,67,150,8.0,900,NaN,Sea,0.001,1,1
2,3,Singapore Port,Malaysia Port,34,50,4.0,650,NaN,Sea,0.000,1,0
3,4,Malaysia Port,Singapore Port,34,50,4.0,650,NaN,Sea,0.001,1,0
4,5,Gujarat Port,Malaysia Port,67,300,7.0,730,NaN,Sea,0.000,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
345,46,Gujarat Railway Station,Gujarat Warehouse,34,100,1.0,0,NaN,Truck,0.000,7,1
346,47,Singapore Warehouse,Malaysia Warehouse,34,150,1.0,0,20.0,Truck,0.000,7,1
347,48,Malaysia Warehouse,Singapore Warehouse,34,150,1.0,0,10.0,Truck,0.001,7,1
348,49,Gujarat Warehouse,Mumbai Warehouse,34,125,1.0,0,15.0,Truck,0.000,7,1


order variable shows quoted order. Please check model.xlsx file [order information sheet]

In [ ]:
order

,Order Number,Ship From,Ship To,Commodity,Order Value,Weight (KG),Volume,Shipper Name,Shipper Address,Shipper Country,Consignee Country,Order Date,Required Delivery Date,Journey Type,Tax Percentage
0,1,Singapore Warehouse,Mumbai Warehouse,Honey,50000,21000,34,YCH,8 Bulim Ave,Singapore,India,2022-08-02,2022-08-25,International,0.03
1,2,Malaysia Warehouse,Gujarat Warehouse,Tea,10000,20000,67,YCH,8 Bulim Ave,Malaysia,India,2022-08-02,2022-08-23,International,0.01
2,3,Singapore Warehouse,Gujarat Warehouse,Banana,12000,20000,67,YCH,8 Bulim Ave,Singapore,India,2022-08-03,2022-08-23,International,0.01
3,4,Singapore Warehouse,Gujarat Warehouse,Seed,800000,20,7,YCH,8 Bulim Ave,Singapore,India,2022-08-04,2022-08-24,International,0.10
4,5,Mumbai Warehouse,Malaysia Warehouse,Mango,700000,5000,50,YCH,8 Bulim Ave,India,Malaysia,2022-08-05,2022-08-22,International,0.15
5,6,Gujarat Warehouse,Singapore Warehouse,Apple,30000,25000,67,YCH,8 Bulim Ave,India,Singapore,2022-08-06,2022-08-23,International,0.01
6,7,Malaysia Warehouse,Singapore Warehouse,Litchi,10000,10000,34,YCH,8 Bulim Ave,Malaysia,Singapore,2022-08-07,2022-08-24,International,0.01
7,8,Mumbai Warehouse,Gujarat Warehouse,Coffee,10000,20000,67,YCH,8 Bulim Ave,India,India,2022-08-08,2022-08-25,Domestic,0.00


Resultant solution after optimisation through cvxpy solver. Please check solution.txt file for optimal route, load and cost.

In [ ]:
m.get_output_()

(198484.0,
 {'Goods-1': [('Singapore Warehouse', 'Malaysia Warehouse', '2022-08-02', 0),
   ('Malaysia Warehouse', 'Malaysia Port', '2022-08-05', 0),
   ('Malaysia Port', 'Gujarat Port', '2022-08-06', 0),
   ('Gujarat Port', 'Gujarat Warehouse', '2022-08-13', 0),
   ('Gujarat Warehouse', 'Mumbai Warehouse', '2022-08-14', 0)],
  'Goods-2': [('Malaysia Warehouse', 'Malaysia Port', '2022-08-02', 1),
   ('Malaysia Port', 'Gujarat Port', '2022-08-03', 1),
   ('Gujarat Port', 'Gujarat Warehouse', '2022-08-10', 1)],
  'Goods-3': [('Singapore Warehouse', 'Malaysia Warehouse', '2022-08-03', 2),
   ('Malaysia Warehouse', 'Malaysia Port', '2022-08-05', 2),
   ('Malaysia Port', 'Gujarat Port', '2022-08-06', 2),
   ('Gujarat Port', 'Gujarat Warehouse', '2022-08-13', 2)],
  'Goods-4': [('Singapore Warehouse', 'Malaysia Warehouse', '2022-08-04', 3),
   ('Malaysia Warehouse', 'Malaysia Port', '2022-08-05', 3),
   ('Malaysia Port', 'Gujarat Port', '2022-08-06', 3),
   ('Gujarat Port', 'Gujarat Warehous

In [ ]:
!pip install geopy
import geopy
from geopy.geocoders import Nominatim
import folium

# this is just for demonstration

geolocator = Nominatim(timeout=10, user_agent = "my-application")
location1 = geolocator.geocode('Gujarat Port')
location2 = geolocator.geocode('Mumbai Port')
print(location1)
print((location1.latitude, location1.longitude))
print(location2)
print((location2.latitude, location2.longitude))


p = folium.Map(location=[location1.latitude, location1.longitude],
              zoom_start=15)

loc = [(location1.latitude, location1.longitude),
       (location2.latitude, location2.longitude)]    

folium.PolyLine(loc,
                color='red',
                weight=5,
                opacity=0.8).add_to(p)
p

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Port, Salaya, Khambhalia Taluka, Devbhumi Dwaraka District, Gujarat, India
(22.318400699999998, 69.60041945843892)
Jawaharlal Nehru Port, Sheva, Mumbai Metropolitan Region, Uran, Raigad, Maharashtra, India
(18.945030600000003, 72.94942299008869)


Please check **solution.txt file** for optimal route and load for concerned order.  Data used for simulation are in model.xlsx file.

Solution

Number of Goods: 8

Total cost: 198484.0

Transportation cost: 6125.0

Warehouse cost: 2755.0

Tax cost: 189604.0

------------------------------------

Goods-1  Category: Honey

Start date: 2022-08-02

Arrival date: 2022-08-15

Route:

(1)Date: 2022-08-02  From: Singapore Warehouse  To: Malaysia Warehouse  By: Truck

(2)Date: 2022-08-05  From: Malaysia Warehouse  To: Malaysia Port  By: Truck

(3)Date: 2022-08-06  From: Malaysia Port  To: Gujarat Port  By: Sea

(4)Date: 2022-08-13  From: Gujarat Port  To: Gujarat Warehouse  By: Truck

(5)Date: 2022-08-14  From: Gujarat Warehouse  To: Mumbai Warehouse  By: Truck

------------------------------------

Goods-2  Category: Tea

Start date: 2022-08-02

Arrival date: 2022-08-11

Route:

(1)Date: 2022-08-02  From: Malaysia Warehouse  To: Malaysia Port  By: Truck

(2)Date: 2022-08-03  From: Malaysia Port  To: Gujarat Port  By: Sea

(3)Date: 2022-08-10  From: Gujarat Port  To: Gujarat Warehouse  By: Truck

------------------------------------

Goods-3  Category: Banana

Start date: 2022-08-03

Arrival date: 2022-08-14

Route:

(1)Date: 2022-08-03  From: Singapore Warehouse  To: Malaysia Warehouse  By: Truck

(2)Date: 2022-08-05  From: Malaysia Warehouse  To: Malaysia Port  By: Truck

(3)Date: 2022-08-06  From: Malaysia Port  To: Gujarat Port  By: Sea

(4)Date: 2022-08-13  From: Gujarat Port  To: Gujarat Warehouse  By: Truck

------------------------------------

Goods-4  Category: Seed

Start date: 2022-08-04

Arrival date: 2022-08-14

Route:

(1)Date: 2022-08-04  From: Singapore Warehouse  To: Malaysia Warehouse  By: Truck

(2)Date: 2022-08-05  From: Malaysia Warehouse  To: Malaysia Port  By: Truck

(3)Date: 2022-08-06  From: Malaysia Port  To: Gujarat Port  By: Sea

(4)Date: 2022-08-13  From: Gujarat Port  To: Gujarat Warehouse  By: Truck

------------------------------------
Goods-5  Category: Mango


Start date: 2022-08-05

Arrival date: 2022-08-18

Route:

(1)Date: 2022-08-06  From: Mumbai Warehouse  To: Gujarat Warehouse  By: Truck

(2)Date: 2022-08-07  From: Gujarat Warehouse  To: Gujarat Port  By: Truck

(3)Date: 2022-08-08  From: Gujarat Port  To: Singapore Port  By: Sea

(4)Date: 2022-08-16  From: Singapore Port  To: Singapore Warehouse  By: Truck

(5)Date: 2022-08-17  From: Singapore Warehouse  To: Malaysia Warehouse  By: Truck

------------------------------------

Goods-6  Category: Apple

Start date: 2022-08-06

Arrival date: 2022-08-17

Route:

(1)Date: 2022-08-07  From: Gujarat Warehouse  To: Gujarat Port  By: Truck

(2)Date: 2022-08-08  From: Gujarat Port  To: Singapore Port  By: Sea

(3)Date: 2022-08-16  From: Singapore Port  To: Singapore Warehouse  By: Truck

------------------------------------

Goods-7  Category: Litchi

Start date: 2022-08-07

Arrival date: 2022-08-08

Route:

(1)Date: 2022-08-07  From: Malaysia Warehouse  To: Singapore Warehouse  By: Truck

------------------------------------

Goods-8  Category: Coffee

Start date: 2022-08-08

Arrival date: 2022-08-09

Route:

(1)Date: 2022-08-08  From: Mumbai Warehouse  To: Gujarat Warehouse  By: Truck